In [1]:
import findspark
findspark.init('/home/yerke/spark-3.3.0-bin-hadoop3')

In [34]:
import warnings
warnings.filterwarnings('ignore')
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [30]:
from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml.evaluation import ClusteringEvaluator

In [16]:
dataset = spark.read.format('libsvm').load('../course_data/sample_kmeans_data.txt')

In [17]:
dataset.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|           (3,[],[])|
|  1.0|(3,[0,1,2],[0.1,0...|
|  2.0|(3,[0,1,2],[0.2,0...|
|  3.0|(3,[0,1,2],[9.0,9...|
|  4.0|(3,[0,1,2],[9.1,9...|
|  5.0|(3,[0,1,2],[9.2,9...|
+-----+--------------------+



In [18]:
final_data = dataset.select('features')

In [19]:
final_data.show()

+--------------------+
|            features|
+--------------------+
|           (3,[],[])|
|(3,[0,1,2],[0.1,0...|
|(3,[0,1,2],[0.2,0...|
|(3,[0,1,2],[9.0,9...|
|(3,[0,1,2],[9.1,9...|
|(3,[0,1,2],[9.2,9...|
+--------------------+



In [31]:
kmeans = BisectingKMeans().setK(2).setSeed(1)

In [32]:
model = kmeans.fit(final_data)

In [35]:
model.computeCost(final_data)

0.11999999999994547

In [23]:
predictions = model.transform(final_data)

In [28]:
predictions.show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|           (3,[],[])|         1|
|(3,[0,1,2],[0.1,0...|         1|
|(3,[0,1,2],[0.2,0...|         1|
|(3,[0,1,2],[9.0,9...|         0|
|(3,[0,1,2],[9.1,9...|         0|
|(3,[0,1,2],[9.2,9...|         0|
+--------------------+----------+



In [36]:
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

Silhouette with squared euclidean distance = 0.9997530305375207


In [37]:
model.clusterCenters()

[array([0.1, 0.1, 0.1]), array([9.1, 9.1, 9.1])]